In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-greentaxi-trip-prediction', tags={}>]

## Get the Top 5 runs of an experiment

In [18]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids ='1',
    filter_string = "metrics.rmse < 7",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by = ["metrics.rmse ASC"]
)

In [19]:
for run in runs:
    print(f"Run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

Run id: 1f0e6ede61ca42aeb5824d483d0ccc6a, rmse: 6.3471
Run id: 7305d65588ac4289ae3b5d3142b74a35, rmse: 6.3471
Run id: 180e67d448e94e1096b860f0b266941f, rmse: 6.3471
Run id: f8e4417824574d6aba41af08052c054a, rmse: 6.3513
Run id: c63085561ffe4bdeb0303cc4422c9c8a, rmse: 6.3520


## Load model (not neccesary to interact with the API)

### Register a model

In [21]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [22]:
run_id = "1f0e6ede61ca42aeb5824d483d0ccc6a"
model_name = "nyc-green-taxi-duration-regressor"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'nyc-green-taxi-duration-regressor' already exists. Creating a new version of this model...
2022/05/29 22:54:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-green-taxi-duration-regressor, version 3
Created version '3' of model 'nyc-green-taxi-duration-regressor'.


<ModelVersion: creation_timestamp=1653857697746, current_stage='None', description=None, last_updated_timestamp=1653857697746, name='nyc-green-taxi-duration-regressor', run_id='1f0e6ede61ca42aeb5824d483d0ccc6a', run_link=None, source='./mlruns/1/1f0e6ede61ca42aeb5824d483d0ccc6a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Move the different model between the environments

In [28]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1653857463649, description='', last_updated_timestamp=1653857780047, latest_versions=[<ModelVersion: creation_timestamp=1653857492745, current_stage='Staging', description='', last_updated_timestamp=1653857780047, name='nyc-green-taxi-duration-regressor', run_id='7305d65588ac4289ae3b5d3142b74a35', run_link='', source='./mlruns/1/7305d65588ac4289ae3b5d3142b74a35/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <ModelVersion: creation_timestamp=1653857697746, current_stage='None', description=None, last_updated_timestamp=1653857697746, name='nyc-green-taxi-duration-regressor', run_id='1f0e6ede61ca42aeb5824d483d0ccc6a', run_link=None, source='./mlruns/1/1f0e6ede61ca42aeb5824d483d0ccc6a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-green-taxi-duration-regressor', tags={}>]

In [31]:
lates_versions = client.get_latest_versions(name=model_name)

for version in lates_versions:
    print(f"Version: {version.version}, Environment: {version.current_stage}")

Version: 2, Environment: Staging
Version: 3, Environment: None


In [41]:
model_version = 3
env = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=version,
    stage=env,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653857697746, current_stage='Production', description='The model version 3 was transitioned to Production on 2022-05-29.', last_updated_timestamp=1653858598073, name='nyc-green-taxi-duration-regressor', run_id='1f0e6ede61ca42aeb5824d483d0ccc6a', run_link=None, source='./mlruns/1/1f0e6ede61ca42aeb5824d483d0ccc6a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [37]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name = model_name,
    version = model_version,
    description = f"The model version {model_version} was transitioned to {env} on {date}."
)

<ModelVersion: creation_timestamp=1653857697746, current_stage='Production', description='The model version 3 was transitioned to Production on 2022-05-29.', last_updated_timestamp=1653858547412, name='nyc-green-taxi-duration-regressor', run_id='1f0e6ede61ca42aeb5824d483d0ccc6a', run_link=None, source='./mlruns/1/1f0e6ede61ca42aeb5824d483d0ccc6a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Annotate model